In [1]:
import detection as eddy
import oceanspy as ospy
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cmocean.cm as cmo
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from matplotlib.path import Path
from matplotlib.patches import PathPatch
from matplotlib.colors import Normalize
from matplotlib.collections import LineCollection
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
from matplotlib.colors import SymLogNorm
import tqdm
import ast

In [2]:
# Enable/disable computation of different domains
config_parent_domain = False
config_child_domain = False

In [3]:
# Import data
df_OW = xr.open_dataset('/nird/projects/NS9608K/MSc_EK/Data/Horizontal_Okubo_Weiss.nc')['Okubo_Weiss']
df_OW = df_OW*(1/(10**(-9)))
df_eta = xr.open_dataset('/nird/projects/NS9608K/MSc_EK/Data/Horizontal_Eta.nc')['Eta']
df = xr.open_dataset(r'/nird/projects/NS9608K/MSc_EK/Data/Background_var.nc')
df_hor_vel = xr.open_dataset('/nird/projects/NS9608K/MSc_EK/Data/Horizontal_hor_vel.nc')['hor_vel']
display(df)

<xarray.Dataset>
Dimensions:    (Z: 216, Zp1: 217, Zu: 216, Zl: 216, X: 765, Y: 628, Xp1: 766,
                Yp1: 629, time: 12, time_midp: 11)
Coordinates: (12/18)
  * Z          (Z) float64 -1.0 -3.5 -7.0 ... -3.904e+03 -3.922e+03 -3.942e+03
  * Zp1        (Zp1) float64 0.0 -2.0 -5.0 ... -3.913e+03 -3.932e+03 -3.951e+03
  * Zu         (Zu) float64 -2.0 -5.0 -9.0 ... -3.913e+03 -3.932e+03 -3.951e+03
  * Zl         (Zl) float64 0.0 -2.0 -5.0 ... -3.894e+03 -3.913e+03 -3.932e+03
  * X          (X) float64 -29.92 -29.84 -29.76 -29.67 ... 11.72 11.82 11.92
  * Y          (Y) float64 65.01 65.03 65.04 65.06 ... 78.67 78.75 78.83 78.92
    ...         ...
  * time       (time) datetime64[ns] 2017-09-01 2017-10-01 ... 2018-08-01
    YU         (Y, Xp1) float64 ...
    XU         (Y, Xp1) float64 ...
    YV         (Yp1, X) float64 ...
    XV         (Yp1, X) float64 ...
  * time_midp  (time_midp) datetime64[ns] 2017-09-16 ... 2018-07-16T12:00:00
Data variables:
    U          (time, Z, Y, Xp1) float64 ...
    V          (time, Z, Yp1, X) float64 ...
    Temp       (time, Z, Y, X) float64 ...
    S          (time, Z, Y, X) float64 ...
    Eta        (time, Y, X) float64 ...
    MXLDEPTH   (time, Y, X) float64 ...
    SIarea     (time, Y, X) float64 ...
    SIheff     (time, Y, X) float64 ...
Attributes: (12/24)
    MITgcm_version:          checkpoint67l
    build_user:              malmans2@jhu.edu
    build_host:              compute0673
    build_date:              Fri Nov  8 01:36:45 EST 2019
    MITgcm_URL:              http://mitgcm.org
    MITgcm_tag_id:           
    ...                      ...
    OceanSpy_parameters:     {'rSphere': 6371.0, 'eq_state': 'mdjwf', 'rho0':...
    OceanSpy_name:           IGPyearlong
    OceanSpy_description:    High-resolution numerical simulation carried out...
    OceanSpy_projection:     Mercator(**{})
    OceanSpy_grid_coords:    {'Y': {'Y': None, 'Yp1': 0.5}, 'X': {'X': None, ...
    OceanSpy_grid_periodic:  []

In [4]:
# Inner domain
domain_center = [[-20,0],[71,74]]
OW_center = df_OW.sel(X=slice(domain_center[0][0],domain_center[0][1])).sel(Y=slice(domain_center[1][0],domain_center[1][1])).sel(Z=-1)
eta_center = df_eta.sel(X=slice(domain_center[0][0],domain_center[0][1])).sel(Y=slice(domain_center[1][0],domain_center[1][1]))

# Outer domain
eta = df_eta.sel(X=slice(domain_center[0][0]-2,domain_center[0][1]+2)).sel(Y=slice(domain_center[1][0]-1,domain_center[1][1]+1))
OW = df_OW.sel(X=slice(domain_center[0][0]-2,domain_center[0][1]+2)).sel(Y=slice(domain_center[1][0]-1,domain_center[1][1]+1)).sel(Z=-1)

In [5]:
# Resample data
eta = eta.resample(time='D').mean(dim='time')
eta_center = eta_center.resample(time='D').mean(dim='time')
OW = OW.resample(time='D').mean(dim='time')
OW_center = OW_center.resample(time='D').mean(dim='time')
hor_vel = df_hor_vel.resample(time='D').mean(dim='time')

# Define time extent
time = ['2018-01-01T00:00:00.000000000','2018-01-31T00:00:00.000000000']
eta = eta.sel(time=slice(time[0],time[1]))
eta_center = eta_center.sel(time=slice(time[0],time[1]))
OW = OW.sel(time=slice(time[0],time[1]))
OW_center = OW_center.sel(time=slice(time[0],time[1]))
hor_vel = hor_vel.sel(X=slice(domain_center[0][0]-2,domain_center[0][1]+2)).sel(Y=slice(domain_center[1][0]-2,domain_center[1][1]+2)).sel(Z=-1).sel(time=slice(time[0],time[1]))

# Eddy centerpoints (Child domain)

In [6]:
# Computes or loads eddy centerpoints
run = config_child_domain

if run:
    eddyLocation = []
    print('###Applying filter###')
    for i in range(len(eta)):
        eddyLocation.append(eddy.eddyDetection(eta_center[i],OW_center[i]))
        print('Processing: ',100*(i+1)/len(eta_center), '%')
    with open("eddyCenterpoints_test.txt",'w') as f:
        for time in range(len(eddyLocation)):
            if time != len(eddyLocation)-1:
                f.write(str(eddyLocation[time])+',')
            else:
                f.write(str(eddyLocation[time]))

else:
    print('###Loading previously saved eddy centerpoints###')
    with open("eddyCenterpoints_test.txt",'r') as f:
        data = f.read()
        eddyLocation = ast.literal_eval(data)

###Loading previously saved eddy centerpoints###


# Try updated Eddy center detection

In [ ]:
import EddyDetectionV2 as testeddy # Works much better

In [12]:
test = testeddy.eddyDetection(eta_center[0],OW_center[0])

Generating Frames:   0%|          | 0/131 [00:00<?, ?it/s]

Generating Frames: 100%|██████████| 131/131 [00:00<00:00, 1330.81it/s]

Skipped windows %: 88.18671004880491


In [14]:
display(test[0] == eddyLocation[0][0])
display(test[0],eddyLocation[0][0])

False

[[71.13324712000001, -14.447136549999986],
 [71.13324712000001, -9.397923165000002],
 [71.18737249499999, -13.91529619],
 [71.27832667499999, -5.971855255000001],
 [71.35178564499999, -8.006987669999997],
 [71.35178564499999, -3.6746734799999956],
 [71.38875489499999, -17.184090635],
 [71.463187775, -11.039915429999994],
 [71.51945712999999, -9.484479565000003],
 [71.671460565, -12.905644134999998],
 [71.69066815, -14.714621209999985],
 [71.78742728, -4.4686678699999955],
 [71.88543071000001, -17.419766980000006],
 [71.88543071000001, -8.181655034999997],
 [71.924993885, -10.737527224999994],
 [71.94485499000001, -13.91529619],
 [71.96476979, -13.080454489999997],
 [72.00476254, -10.176089865000002],
 [72.02484154, -11.645367309999994],
 [72.085415495, -13.827028024999999],
 [72.146506485, -1.687209394999996],
 [72.187528595, -16.438886585000006],
 [72.187528595, -8.530200194999997],
 [72.20812984, -16.34663061500001],
 [72.29115009, -13.431002524999997],
 [72.312062055, -14.0036658699

[[71.09734426, -8.399611769999996],
 [71.13324712000001, -14.447136549999986],
 [71.13324712000001, -9.397923165000002],
 [71.13324712000001, -8.268886084999997],
 [71.151252435, -8.138014344999997],
 [71.18737249499999, -13.91529619],
 [71.27832667499999, -5.971855255000001],
 [71.35178564499999, -8.006987669999997],
 [71.35178564499999, -3.6746734799999956],
 [71.38875489499999, -17.184090635],
 [71.463187775, -11.039915429999994],
 [71.51945712999999, -9.484479565000003],
 [71.671460565, -12.905644134999998],
 [71.69066815, -14.714621209999985],
 [71.78742728, -4.4686678699999955],
 [71.88543071000001, -17.419766980000006],
 [71.88543071000001, -8.181655034999997],
 [71.924993885, -10.737527224999994],
 [71.94485499000001, -13.91529619],
 [71.96476979, -13.080454489999997],
 [72.00476254, -10.176089865000002],
 [72.02484154, -11.645367309999994],
 [72.085415495, -13.827028024999999],
 [72.146506485, -1.687209394999996],
 [72.187528595, -16.438886585000006],
 [72.187528595, -8.530200

In [ ]:
# Computes or loads eddy centerpoints
run = config_child_domain
run = True

if run:
    eddyLocation = []
    print('###Applying filter###')
    for i in range(len(eta)):
        eddyLocation.append(testeddy.eddyDetection(eta_center[i],OW_center[i]))
        print('Processing: ',100*(i+1)/len(eta_center), '%')
    with open("eddyCenterpoints_test_v2.txt",'w') as f:
        for time in range(len(eddyLocation)):
            if time != len(eddyLocation)-1:
                f.write(str(eddyLocation[time])+',')
            else:
                f.write(str(eddyLocation[time]))

else:
    print('###Loading previously saved eddy centerpoints###')
    with open("eddyCenterpoints_test_v2.txt",'r') as f:
        data = f.read()
        eddyLocation = ast.literal_eval(data)

###Applying filter###


Generating Frames: 100%|██████████| 131/131 [00:00<00:00, 1335.72it/s]


Skipped windows %: 88.18671004880491
Processing:  3.225806451612903 %


Generating Frames: 100%|██████████| 131/131 [00:00<00:00, 1424.77it/s]


Skipped windows %: 88.674759103992
Processing:  6.451612903225806 %


...

Skipped windows %: 85.90199688935766
Processing:  93.54838709677419 %


Generating Frames: 100%|██████████| 131/131 [00:00<00:00, 1236.24it/s]


Skipped windows %: 86.90848633283873
Processing:  96.7741935483871 %


Generating Frames: 100%|██████████| 131/131 [00:00<00:00, 1293.72it/s]

Skipped windows %: 87.36256860396517
Processing:  100.0 %
